In [28]:
from datasets import load_dataset
from utils import jpeg_compress, webp_compress, pad, crop, nn_compress, hific_lo_compress
import compressai
# import PIL
# import torchaudio
# import evaluate
from evaluate import evaluator
from transformers import pipeline
# from io import BytesIO
# import encodec
import torch
# import torch.nn.functional as F
# import numpy as np
# import compressai.zoo
# from torchvision import transforms
# import matplotlib.pyplot as plt
# from transformers import AutoFeatureExtractor, SegformerForSemanticSegmentation
# from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation
# import opuspy
# import tempfile
# import os
# import tfci

In [2]:
device = "cuda"
net_mbt2018 = compressai.zoo.mbt2018(quality=1, pretrained=True).eval().to(device)
def mbt2018_compress(sample):
    return nn_compress(sample,net_mbt2018,device)

In [25]:
image_compression_methods = [
    jpeg_compress,
    webp_compress,
    mbt2018_compress,
    hific_lo_compress
]
models = [
    "microsoft/resnet-18",
    "google/vit-base-patch16-224",
    "microsoft/beit-large-patch16-224"
]

In [4]:
imagenet = [load_dataset("imagenet-1k", split="validation[:1000]")]

In [5]:
for method in image_compression_methods:
    imagenet.append(imagenet[0].map(method))

In [26]:
pipe = []
for model in models:
    pipe.append(
        pipeline(
            task="image-classification",
            model="google/vit-base-patch16-224",
            device="cuda:0"
        )
    )

In [27]:
task_evaluator = evaluator("image-classification")

In [32]:
eval_results = []
for i_method,method in enumerate(imagenet):
    eval_results.append([])
    for i_model,model in enumerate(pipe):
        eval_results[i_method].append(
            task_evaluator.compute(
                model_or_pipeline=model,
                data=method,
                metric="accuracy",
                label_mapping=model.model.config.label2id
            )
        )

In [44]:
[[rij['accuracy'] for rij in r] for r in eval_results]

[[0.799, 0.799, 0.799],
 [0.639, 0.639, 0.639],
 [0.72, 0.72, 0.72],
 [0.664, 0.664, 0.664],
 [0.796, 0.796, 0.796]]